# Newton$-$Raphson fractal for arbitrary functions

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import colormaps

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
# Source of color: https://hextoral.com/hex-color/F4F0E8/dunn-edwards/
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(rc=custom_settings)

## Define a grid of points

In [ ]:
def get_even_points(N, limx, limy):
    '''
    Get the number of points along the X and Y dimensions of an
    arbitrary 2D grid, where the points are evenly distributed.

    Parameters
    ----------
    N : float
        Approximate number of points along the shorter side of the grid.
    limx : tuple
        The left and right limits of the border along the X dimension.
    limy : tuple
        The left and right limits of the border along the Y dimension.

    Returns
    -------
    (Nx, Ny) : tuple of ints
        The number of points along the X and Y dimension, respectively.
    '''
    dx = (limx[1]-limx[0]) / (limy[1]-limy[0])
    Nx, Ny = int(round(N * max(dx, 1))), int(round(N * max(1/dx, 1)))
    return Nx, Ny

In [ ]:
def get_grid(Nx, Ny, limx, limy):
    '''
    Generate a grid of points on the Re-Im complex space in an arbitrary
    box size.
    '''
    X = np.meshgrid(np.linspace(*limx, Nx), np.linspace(*limy, Ny))
    X = (X[0] + X[1]*1j).flatten()
    return X

In [ ]:
N = 100
limx, limy = (-5, 5), (-2.5, 2.5)
Nx, Ny = get_even_points(N, limx, limy)
G = get_grid(Nx, Ny, limx, limy)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.set_xlim(limx)
ax.set_ylim(limy)
ax.scatter(G.real, G.imag, color='0.3', s=1)
plt.show()

## Find closest roots to points

In [ ]:
def closest_roots(R, X):
    '''
    Find the index of the closest root from a set of R roots to a given
    X value(s).
    '''
    return np.argmin([np.abs(X - r) for r in R], axis=0)

In [ ]:
def nr_colors(R, X, cmap=cm.viridis):
    '''
    Generate a list of color values sampled from a colormap for a given
    X value(s).

    Parameters
    ----------
    R : list or array-like
        A list of roots to check the neighbour relations to.
    X : float or array-like
        A value or an array of values to get color values for.
    cmap : Callable or `~matplotlib.colors.Colormap`, default 'viridis'
        A colormap instance to sample values from or a function to get
        color values from.

    Returns
    -------
    list
        A list of RGBA color values sampled from a given colormap, each
        corresponding to the element(s) in the input X.
    '''
    return cmap((r:=closest_roots(R, X))/max(r))

In [ ]:
# Example usage with a specific function and derivative
F = lambda z: np.cos(z)
D = lambda z: -np.sin(z)
R = np.pi * np.arange(-5, 5)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.set_xlim(limx)
ax.set_ylim(limy)
ax.scatter(G.real, G.imag, c=nr_colors(R, G), s=1)
plt.show()

## Apply Newton$-$Raphson method on a grid of points

In [ ]:
from functools import reduce

In [ ]:
def nr_step(F, D, X):
    '''
    Calculate a single step of the Newton-Raphson iterative method on
    a polynomial, evaluated at given X value(s).

    Parameters
    ----------
    P : newton.polynomial.Polynomial
        A polynomial instance to apply the Newton-Raphson iterative
        method on.
    X : float or array-like
        A value or an array of values to evaluate the input polynomial
        at.
    '''
    return np.nan_to_num(X - F(X)/D(X))

In [ ]:
def nr_iter(F, D, X, N=20):
    '''
    Calculate multiple consecutive steps of the Newton-Raphson iterative
    method on a polynomial, evaluated at given X value(s).

    Parameters
    ----------
    P : newton.polynomial.Polynomial
        A polynomial instance to apply the Newton-Raphson iterative
        method on.
    X : float or array-like
        A value or an array of values to evaluate the input polynomial
        at.
    N : int, default=20
        Number of iteration to take.
    '''
    return reduce(lambda x, _: nr_step(F, D, x), range(N), X)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.axis(False)
ax.grid(False)
ax.imshow(closest_roots(R, nr_iter(F, D, G, 3)).reshape(Ny, Nx), cmap=cm.gnuplot2)
plt.show()

## Connect everything above

In [ ]:
F = lambda z: np.nan_to_num(np.cos(z))
D = lambda z: np.nan_to_num(-np.sin(z))
R = np.pi * np.arange(-5, 5)

In [ ]:
N = 4096
c, r = (0, 0), (1, 3)
limx, limy = (c[0]-r[0]/2, c[0]+r[0]/2), (c[1]-r[1]/2, c[1]+r[1]/2)
Nx, Ny = get_even_points(N, limx, limy)
G = get_grid(Nx, Ny, limx, limy)

In [ ]:
%%time
Gn = closest_roots(R, nr_iter(F, D, G, 100)).reshape(Ny, Nx)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15), facecolor='black')
ax.axis(False)
ax.grid(False)
ax.imshow(Gn.T, cmap=cm.Spectral)
plt.savefig('cos_z.png', dpi=1200, bbox_inches='tight', pad_inches=0.01)
plt.show()